## API

In [4]:
from fastapi import FastAPI
import pandas as pd
import time
import datetime
import re
import requests
import json
from bs4 import BeautifulSoup

app = FastAPI()


@app.get("/parse")
def parsing():
    return json.dumps(_parsing())


def _parsing():
    articles = []
    articles += parse_data_bukhonline()
    articles += parse_data_rbc()
    articles += parse_data_lenta()
    return articles


def parse_data_rbc():
    articles_dict = {}
    cur_time = round(time.time())
    for i in range(3):
        try:
            calc_time = cur_time - i * 86400
            url = f'https://www.rbc.ru/v10/ajax/get-news-feed-short/project/rbcnews.uploaded/lastDate/{calc_time}/limit/22'
            body = requests.get(url)
            raw_body_list = json.loads(body.text)['items']

            for body in raw_body_list:
                soup = BeautifulSoup(body['html'], "html.parser")
                try:
                    article = {
                        'title': soup.find_all('span')[2].text.replace('  ', '').strip('\n'),
                        'link': re.findall(r'href=\"(.*)\"', str(soup.find_all('a')[1]))[0],
                    }
                    articles_dict[re.findall(r'id=\"(.*)\"', str(soup))[0]] = article
                except:
                    continue
        except Exception as e:
            print(e)
            break

    articles = []
    for article in articles_dict.values():
        article_body = requests.get(article['link'])
        article_soup = BeautifulSoup(article_body.text, "html.parser")
        article['full_info'] = article_soup.find_all('p')[0].text
        try:
            article['datetime'] = re.findall(r'datetime=\"(.*)\"', str(article_soup.find_all('time')[0]))[0].split('+')[
                0]
        except:
            article['datetime'] = None
        articles.append(article)

    return articles


def parse_data_lenta():
    articles = []
    today = datetime.datetime.now()
    cur_date = datetime.datetime.now() - datetime.timedelta(days=3)
    while today.day != cur_date.day or today.month != cur_date.month or today.year != cur_date.year:
        month = cur_date.month if len(str(cur_date.month)) == 2 else f'0{cur_date.month}'
        day = cur_date.day if len(str(cur_date.day)) == 2 else f'0{cur_date.day}'
        url = f'https://lenta.ru/{cur_date.year}/{month}/{day}/'
        print(url)
        lenta_page = requests.get(url)
        lenta_soup = BeautifulSoup(lenta_page.text, "html.parser")
        arch_articles = lenta_soup.find_all('ul')[3].find_all('li')
        for article in arch_articles:
            try:
                link = re.findall(r'href=\"(.*)\"', str(article))[0].split('\"')[0]
                if not link.startswith('https://lenta.ru'):
                    link = 'https://lenta.ru' + link
                parsed_dates = re.findall(r'\d{4}\/\d{2}\/\d{2}', link)[0].split('\"')[0].split('/')
            except:
                continue
            try:
                art_json = {
                    'title': article.find_all('span')[0].text,
                    'link': link,
                    'datetime': datetime.datetime(year=int(parsed_dates[0]), month=int(parsed_dates[1]),
                                                  day=int(parsed_dates[2]))
                }
                articles.append(art_json)
            except:
                continue
        cur_date += datetime.timedelta(days=1)

    for article in articles:
        art_body = requests.get(article['link'])
        article['full_info'] = BeautifulSoup(art_body.text, "html.parser").text

    return articles

def parse_data_bukhonline():
    request_works = True
    page = 1
    articles = []
    max_pages = 3
    while request_works and page <= max_pages:
        URL = f'https://www.buhonline.ru/ajax/pub/news?lastPublicationWithSameDataIds=&lastPublicationHasImage=false&hasMore=true&page={page}&exceptIds=0&tagId='
        resp = requests.get(URL)
        try:
            jsoned_resp = json.loads(resp.text)
            html = jsoned_resp['html']
            soup = BeautifulSoup(html, "html.parser")
            containers = []
            for div in soup.find_all("div", {'class': 'tile tile_other-publications tile_max-w-700'}):
                containers.append(div)

            for container in containers:
                jsoned_article = {
                    'name': container.find("a").text.replace(u'\xa0', u' '),
                    'link': 'https://www.buhonline.ru' + str(re.findall(r'\"(.*)\"', str(container.find("a")))[0]),
                    'small_info': container.find('p').text.replace(u'\xa0', u' '),
                }
                articles.append(jsoned_article)
            page += 1
        except:
            request_works = False

    for article in articles:
        full_page = requests.get(article['link'])
        soup = BeautifulSoup(full_page.text, "html.parser")
        full_text = ''
        for p in soup.find_all('p'):
            full_text += p.text.replace(u'\xa0', u' ')
        article['datetime'] = re.findall(r'\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}\.\d{7}', str(soup.find_all('time')))[0]
        article['full_info'] = full_text

    return articles


## Parsing

In [5]:
a = parse_data_lenta()
predict_df_lenta = pd.DataFrame(a)

a = parse_data_rbc()
predict_df_rbc = pd.DataFrame(a)

predict_df = pd.concat([predict_df_lenta, predict_df_rbc])
predict_df

https://lenta.ru/2022/10/05/
https://lenta.ru/2022/10/06/
https://lenta.ru/2022/10/07/


,title,link,datetime,full_info
0,Россия первой в мире создала гиперзвуковые ра...,https://lenta.ru/articles/2022/10/05/zvuk/,2022-10-05 00:00:00,«А у нас есть» Россия первой в мире создала ги...
1,Как избавиться от них: 12 лайфхаков от психол...,https://lenta.ru/articles/2022/10/05/mysli/,2022-10-05 00:00:00,Навязчивые мысли о плохом отравляют жизнь и ру...
2,Что стоит за обострением армяно-азербайджанск...,https://lenta.ru/articles/2022/10/05/az_arm/,2022-10-05 00:00:00,Не договорились. Что стоит за обострением армя...
3,«Кольца власти» и «Дом дракона» подходят к фи...,https://lenta.ru/articles/2022/10/05/dragons_v...,2022-10-05 00:00:00,Дракона мать. «Кольца власти» и «Дом дракона» ...
4,Где отдыхала элита СССР и как были устроены в...,https://lenta.ru/articles/2022/10/05/gosdachi/,2022-10-05 00:00:00,Здесь был генсек. Где отдыхала элита СССР и ка...
...,...,...,...,...
61,«Яндекс» объяснил исчезновение с карт границ р...,https://www.rbc.ru/technology_and_media/06/10/...,2022-10-06T17:09:15,Основные сценарии использования карт «Яндекса»...
62,Индексы США выросли после данных по заявкам на...,https://quote.ru/news/article/633ecf509a794775...,2022-10-06T17:07:43,Американские фондовые индексы выросли в начале...
63,Роскомнадзор ограничил доступ к сайту «вселенн...,https://www.rbc.ru/politics/06/10/2022/633ed80...,2022-10-06T17:02:29,Роскомнадзор по требованию Генпрокуратуры огра...
64,Reuters узнал о предложении группы стран ЕС вв...,https://www.rbc.ru/economics/06/10/2022/633ecf...,2022-10-06T16:57:29,"Польша, Италия, Греция и другие страны ЕС подг..."


In [7]:
import nltk
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer

nltk.download('stopwords')
stop_words=stopwords.words("russian")

def cleaning(text):
    text = re.sub(r'[^\w\s]+|[\d]+', r'',text).strip()
    cleaned_text=[]
    morr = MorphAnalyzer()
    tokens_list=text.split()
    for token in tokens_list:
        token_small=token.lower() #converting to lower case

        if token_small not in stop_words:
            cleaned_text.append(morr.parse(token_small)[0].normal_form)
    clean_text=" ".join(cleaned_text)
    return clean_text

def tokenize(text):
    split=re.split("\W+",text) 
    return split

# клининг новых данных
predict_df['clean_info'] = predict_df['full_info'].apply(lambda x: cleaning(x))

[nltk_data] Downloading package stopwords to /home/timur/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
predict_df.head()

,title,link,datetime,full_info,clean_info
0,Россия первой в мире создала гиперзвуковые ра...,https://lenta.ru/articles/2022/10/05/zvuk/,2022-10-05 00:00:00,«А у нас есть» Россия первой в мире создала ги...,россия первый мир создать гиперзвуковой ракета...
1,Как избавиться от них: 12 лайфхаков от психол...,https://lenta.ru/articles/2022/10/05/mysli/,2022-10-05 00:00:00,Навязчивые мысли о плохом отравляют жизнь и ру...,навязчивый мысль плохой отравлять жизнь рушить...
2,Что стоит за обострением армяно-азербайджанск...,https://lenta.ru/articles/2022/10/05/az_arm/,2022-10-05 00:00:00,Не договорились. Что стоит за обострением армя...,договориться стоить обострение армяноазербайдж...
3,«Кольца власти» и «Дом дракона» подходят к фи...,https://lenta.ru/articles/2022/10/05/dragons_v...,2022-10-05 00:00:00,Дракона мать. «Кольца власти» и «Дом дракона» ...,дракон мать кольцо власть дом дракон подходить...
4,Где отдыхала элита СССР и как были устроены в...,https://lenta.ru/articles/2022/10/05/gosdachi/,2022-10-05 00:00:00,Здесь был генсек. Где отдыхала элита СССР и ка...,генсек отдыхать элита ссср устроить вилла сове...


## fasttext

In [68]:
import fasttext

In [69]:
model = fasttext.load_model('model/fasttext_model.bin')

In [70]:
predict_df['pred_class'] = predict_df['clean_info'].apply(model.predict)

predict_df['pred_class'] = predict_df['pred_class'].apply(lambda x: str(list(x)[0]))

In [71]:
predict_df['pred_class']= predict_df['pred_class'].apply(lambda x: x[x.rfind('__')+2: x.find(',')-1])

In [72]:
predict_df['pred_class'].value_counts()

other    144
buh       15
Name: pred_class, dtype: int64

In [73]:
text_clean = []
for index, row in predict_df.iterrows():
        text_clean.append(row['clean_info'].split()) # ТУТ НАДО BUH DF

from gensim.models import Phrases
bigram = Phrases(text_clean) # Создаем биграммы на основе корпуса
trigram = Phrases(bigram[text_clean])# Создаем триграммы на основе корпуса

for idx in range(len(text_clean)):
    for token in bigram[text_clean[idx]]:
        if '_' in token:
            # Токен это би грамма, добавим в документ.
            text_clean[idx].append(token)
    for token in trigram[text_clean[idx]]:
        if '_' in token:
            # Токен это триграмма, добавим в документ.
            text_clean[idx].append(token)

In [74]:
from gensim.corpora.dictionary import Dictionary
from numpy import array
dictionary = Dictionary(text_clean)
#dictionary.filter_extremes(no_below=10, no_above=0.1)
#Создадим словарь и корпус для lda модели
corpus = [dictionary.doc2bow(doc) for doc in text_clean]
print('Количество уникальных токенов: %d' % len(dictionary))
print('Количество документов: %d' % len(corpus))

Количество уникальных токенов: 11328
Количество документов: 159


In [75]:
from gensim.models.ldamulticore import LdaMulticore
model=LdaMulticore(corpus=corpus,id2word=dictionary, num_topics=3)

In [80]:
model.save('model/LDA.model')

In [82]:
import pickle

file_name = 'myModel.sav'
pickle.dump(model, open('model/LDA.model', 'wb'))

In [78]:
a = {}
for i in model.show_topics():
    a[float(i[1].split('*')[0])] = i[1].split('"')[1]
sorted(a.values(), reverse=True)[0].capitalize()

'Россия'

In [ ]:
def get_trend('